In [1]:
#SETUP
import os
from openai import OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
# To finetune finetune_truth.jsonl on davinci-002 (the successor of curie, which is now deprecated)
client.files.create(
  file=open("TruthfulQA/data/finetune_truth.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=truth_file_id, 
  model="davinci-002",
  suffix="truthful",
  hyperparameters={
    "n_epochs":5,
    "batch_size":21,
    "learning_rate_multiplier":0.1
  }
)

In [ ]:
# Do the same for finetune_info to obtain GPT-info
client.files.create(
  file=open("TruthfulQA/data/finetune_info.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=truth_file_id, 
  model="davinci-002",
  suffix="informative",
  hyperparameters={
    "n_epochs":5,
    "batch_size":21,
    "learning_rate_multiplier":0.1
  }
)

In [4]:
# Print last 5 fine_tuning jobs
fine_tuning_jobs = client.fine_tuning.jobs.list()
for job in fine_tuning_jobs.data[-5:]:
    print(job)
    # print(f"Job ID: {job.id}, Status: {job.status}, Model: {job.model}, Created At: {job.created_at}")


FineTuningJob(id='ftjob-Y5YvB0WsZfbaIvyl772gSic8', created_at=1719773704, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:davinci-002:harvard-university:truthful:9fudyDuw', finished_at=1719776213, hyperparameters=Hyperparameters(n_epochs=5, batch_size=21, learning_rate_multiplier=0.1), model='davinci-002', object='fine_tuning.job', organization_id='org-nNVoHcNN219zvOhl3QIKCmhd', result_files=['file-NhtZUTYjw87iExB8z1kWxbYs'], seed=1680046744, status='succeeded', trained_tokens=3492660, training_file='file-vIi4AtRospUvLolbrcVmfAGy', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='truthful')
FineTuningJob(id='ftjob-ZeZPpWVeB7ZEBEnAeGqmcyfn', created_at=1719704617, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:davinci-002:harvard-university:informational:9fcfwUfc', finished_at=1719707142, hyperparameters=Hyperparameters(n_epochs=5, batch_size=21, learning_rate_multiplier=0.1), model='davinci-002', object='